<a href="https://colab.research.google.com/github/MariannaMois/ml_sellout_marketplace/blob/master/brand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Унификация брендов для маркетплейса**

Предоставлены исторические данные парсинга о продажах на маркетплейсах. Данный проект универсальный и можно использовать для любых данных о продажах с неполным и неточным описанием брендов.


Унификация брендов включает в себя:
*   поиск брендов в описании, если в разделе бренд наименование не указано
*   лемматизация названий брендов
*   объединение англоязычных и русскоязычных наименований, различных написаний одного и того же бренда на основе содержания наименования в **названии бренда и в тексте**

Цель проекта
Создать автоматизированную систему приведения наименований брендов к единому стандарту на основе:
*   лингвистического анализа
*   машинного обучения (нечеткого сравнения строк)
*   бизнес-правил компании

Проект решает проблему множественных вариантов написания брендов на разных языках, с опечатками и альтернативными обозначениями.


Задачи проекта:

1. Нормализация входных данных

Создание системы трансформаций:
* механизм замен по словарю
* учет исключений и контекстных правил
* интеграция справочника брендов. Справочник составлен вручную аналитиком по бизнес-правилам компании

2. Реализация алгоритмов сравнения:
* использование fuzzy-логики
* лемматизация названий


3. Валидация и тестирование
* оценка точности сопоставления
* анализ ошибок

Технологический стек


| Компонент             | Технологии/Инструменты         |
|-----------------------|--------------------------------|
| Обработка данных      | Pandas, NumPy                 |
| Сравнение строк       | RapidFuzz (fuzz.WRatio)       |
| Работа с Excel        | OpenPyXL                      |
| Лемматизация          | PyMorphy3                     |
| Инфраструктура        | Google Colab                  |


# Подключение и установка библиотек


На данном этапе подключается Google Drive для работы с файлами, а также устанавливаются библиотеки для обработки текста и работы с Excel.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install openpyxl

In [3]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.3 MB/s eta 0:00:00


In [4]:
from google.colab import files
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz

In [5]:
file_path = '/content/drive/MyDrive/time/ДЕКАБРЬ_WB.xlsx'

In [6]:
test = pd.read_excel(file_path, sheet_name='Sheet1')

In [7]:
test

,Фото,Название,SKU,Предмет,Продавец,Бренд,ID склейки,Категория,Выручка,"Продажи, кол-во",...,Дата первого отзыва,brand_old,text,Processed_text,cluster,ГРУППА,ПОДГРУППА,КАТЕГОРИЯ,ПОДКАТЕГОРИЯ,Unnamed: 31
0,https://basket-04.wbbasket.ru/vol515/part51524...,Гель для бровей фиксирующий стойкий прозрачный...,51524888,Красота / Гели для бровей,ООО Дрожь Бьюти,PUSY,188507080,Красота/Макияж/Брови/Гель для бровей,28632331,53841,...,2022-10-21,PUSY,Гель для бровей фиксирующий стойкий прозрачный...,гель бровь фиксировать стойкий прозрачный мл,8.0,ДЕКОР,БРОВИ,Фиксация для бровей,Гель,NaN
1,https://basket-07.wbbasket.ru/vol1105/part1105...,"Тушь для ресниц черная объем и удлинение, 11 г",110572701,Красота / Туши,ООО Космо Бьюти,Lavant,100723030,Красота/Макияж/Глаза/Тушь,14371545,34902,...,2022-10-27,Lavant,"Тушь для ресниц черная объем и удлинение, 11 г",тушь ресница чёрный объём удлинение,16.0,ДЕКОР,ГЛАЗА,Тушь,Тушь,NaN
2,https://basket-14.wbbasket.ru/vol2116/part2116...,Гель для бровей версия 2.0 супер сильная фиксация,211695539,Красота / Гели для бровей,ООО Дрожь Бьюти,PUSY,188507080,Красота/Макияж/Брови/Гель для бровей,12046387,14468,...,2022-10-21,PUSY,Гель для бровей версия 2.0 супер сильная фиксация,гель бровь версия супер сильный фиксация,8.0,ДЕКОР,БРОВИ,Фиксация для бровей,Гель,NaN
3,https://basket-04.wbbasket.ru/vol704/part70492...,Консилер для лица 01 LIGHT,70492070,Красота / Корректоры,ООО Софис,7DAYS,55838453,Красота/Макияж/Лицо/Корректор,11650757,31440,...,2023-01-19,7DAYS,Консилер для лица 01 LIGHT,консилер лицо light,50.0,ДЕКОР,ЛИЦО,Корректор/консилер,Консилер,NaN
4,https://basket-17.wbbasket.ru/vol2643/part2643...,Адвент календарь с косметикой 12 Days,264347880,Красота / Наборы декоративной косметики,ООО Софис,B.COLOUR PROFESSIONAL,225345435,Красота/Макияж/Лицо/Набор декоративной косметики,11495523,2193,...,2024-06-17,B.COLOUR PROFESSIONAL,Адвент календарь с косметикой 12 Days,адвент календарь косметика days,32.0,ДЕКОР,ГУБЫ,Помада,Стик,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98001,https://basket-17.wbbasket.ru/vol2746/part2746...,Карандаш для губ матовый стойкий 768 оттенок,274636628,Красота / Косметические карандаши,ИП Аскарбеков А Н,NaN,270877652,NaN,50,1,...,1970-01-01,Нет бренда,Карандаш для губ матовый стойкий 768 оттенок,карандаш губа матовый стойкий оттенок,37.0,ДЕКОР,ГУБЫ,Карандаш для губ,Автоматический,NaN
98002,https://basket-01.wbbasket.ru/vol89/part8908/8...,Румяна для лица color icon,8908933,Красота / Румяна,АО Единая Европа-Холдинг,Wet n Wild,2646816,NaN,40,1,...,2016-08-10,Wet n Wild,Румяна для лица color icon,румяна лицо color icon,57.0,ДЕКОР,ЛИЦО,Румяна,Румяна компактные,NaN
98003,https://basket-01.wbbasket.ru/vol107/part10791...,Косметический карандаш для губ LP15 тон 07 кла...,10791335,Красота / Косметические карандаши,ООО Евротрейд,LavelleCollection,8162763,NaN,26,1,...,2020-02-15,LavelleCollection,Косметический карандаш для губ LP15 тон 07 кла...,косметический карандаш губа тон классич красный,37.0,ДЕКОР,ГУБЫ,Карандаш для губ,Автоматический,NaN
98004,https://basket-01.wbbasket.ru/vol22/part2268/2...,"Тушь для ресниц ""mega impact"" EC148 тон very b...",2268196,Красота / Туши,АО Единая Европа-Холдинг,Wet n Wild,1967389,NaN,24,1,...,2015-09-04,Wet n Wild,"Тушь для ресниц ""mega impact"" EC148 тон very b...",тушь ресница mega impact тон very black,16.0,ДЕКОР,ГЛАЗА,Тушь,Тушь,NaN


# Предобработка данных и загрузка справочника

В пустых строках проставляется "Нет бренда"

In [8]:
test['Brand'] = test['Бренд'].fillna('НЕТ БРЕНДА')

In [9]:
test['Brand'] = test['Brand'].str.upper()

In [10]:
#загружаем справочник
reference_file_path = '/content/drive/MyDrive/Dekor_NEW/Справочник брендов.xlsx'
reference = pd.read_excel(reference_file_path, sheet_name='Лист1')

Списки слов для:
* удаления (delete)
* переименования (rename) по частичному совпадению
* переименования по точному совпадению (для всей ячейки, name)

In [11]:
delete = pd.read_excel(reference_file_path, sheet_name='Лист3')

In [12]:
name = pd.read_excel(reference_file_path, sheet_name='Лист6')

In [13]:
rename = pd.read_excel(reference_file_path, sheet_name='Лист5')

In [14]:
#список преобразует элементы столбца 'Delete' из "delete" в строки
symbols_to_delete = [str(symbol) for symbol in delete['Delete'].tolist()]

Дублируем столбцы со старым наименованием для проверки правильности преобразований во вновь внесенных позициях в справочнике.

In [15]:
test['Товар_old'] = test['Название']

In [16]:
test['Товар'] = test['Название'].str.upper()

1. Удаляем символы из списка:

In [17]:
# удаление символов (список из справочника) из строки
def remove_symbols(brand, symbols):
    for symbol in symbols:
        brand = brand.replace(symbol, '')
    return brand

In [18]:
test['Brand'] = test['Brand'].apply(lambda x: remove_symbols(x, symbols_to_delete))

2. Заменяем бренды по точному совпадению (для коротких наименований и общеупотребительных слов). Пример таких брендов: MAC, Beauty. Таким бренды нельзя искать в текстовом описании, т.к. это может быть часть рекламного описания.

In [19]:
def replace_brand(test, name, Brand_is, Brand_new):
    test['Brand'] = test['Brand'].apply(lambda x: name[Brand_new] if x == name[Brand_is] else x)

In [20]:
test['Brand'] = test['Brand'].apply(lambda x: name['Brand_new'].iloc[0] if x == name['Brand_is'].iloc[0] else x)

2. Переименовываем бренды по совпадению из списка rename. Поиск по полю, за исключением прописанных в Exceptions:

In [21]:
# Функция для замены значений в столбце Brand на основе данных из rename_df
def replace_brand(brand, product, rename_df):
    # Проход по каждой строке в DataFrame rename_df
    for _, row in rename_df.iterrows():
        # Извлечение значения Brand_loc из текущей строки
        brand_loc = row['Brand_loc']
        # Получение списка исключений (разделенных запятыми) или пустой список, если исключений нет
        exceptions = row['Exceptions'].split(',') if pd.notna(row['Exceptions']) else []

        # Проверка условий:
        # 1. Brand_loc содержится в текущем бренде ИЛИ в названии товара
        # 2. Ни одно из исключений не встречается в бренде или названии товара
        if (brand_loc in brand or brand_loc in product) and \
           not any(exc.strip() in brand or exc.strip() in product for exc in exceptions):
            # Возврат нового значения бренда из текущей строки rename_df
            return row['Brand_new']

    # Если условия не выполнены ни для одной строки, возвращаем исходный бренд
    return brand

In [22]:
test['Brand'] = test.apply(lambda row: replace_brand(str(row['Brand']), str(row['Товар']), rename), axis=1)

In [23]:
test

,Фото,Название,SKU,Предмет,Продавец,Бренд,ID склейки,Категория,Выручка,"Продажи, кол-во",...,Processed_text,cluster,ГРУППА,ПОДГРУППА,КАТЕГОРИЯ,ПОДКАТЕГОРИЯ,Unnamed: 31,Brand,Товар_old,Товар
0,https://basket-04.wbbasket.ru/vol515/part51524...,Гель для бровей фиксирующий стойкий прозрачный...,51524888,Красота / Гели для бровей,ООО Дрожь Бьюти,PUSY,188507080,Красота/Макияж/Брови/Гель для бровей,28632331,53841,...,гель бровь фиксировать стойкий прозрачный мл,8.0,ДЕКОР,БРОВИ,Фиксация для бровей,Гель,NaN,PUSY,Гель для бровей фиксирующий стойкий прозрачный...,ГЕЛЬ ДЛЯ БРОВЕЙ ФИКСИРУЮЩИЙ СТОЙКИЙ ПРОЗРАЧНЫЙ...
1,https://basket-07.wbbasket.ru/vol1105/part1105...,"Тушь для ресниц черная объем и удлинение, 11 г",110572701,Красота / Туши,ООО Космо Бьюти,Lavant,100723030,Красота/Макияж/Глаза/Тушь,14371545,34902,...,тушь ресница чёрный объём удлинение,16.0,ДЕКОР,ГЛАЗА,Тушь,Тушь,NaN,LAVANT,"Тушь для ресниц черная объем и удлинение, 11 г","ТУШЬ ДЛЯ РЕСНИЦ ЧЕРНАЯ ОБЪЕМ И УДЛИНЕНИЕ, 11 Г"
2,https://basket-14.wbbasket.ru/vol2116/part2116...,Гель для бровей версия 2.0 супер сильная фиксация,211695539,Красота / Гели для бровей,ООО Дрожь Бьюти,PUSY,188507080,Красота/Макияж/Брови/Гель для бровей,12046387,14468,...,гель бровь версия супер сильный фиксация,8.0,ДЕКОР,БРОВИ,Фиксация для бровей,Гель,NaN,PUSY,Гель для бровей версия 2.0 супер сильная фиксация,ГЕЛЬ ДЛЯ БРОВЕЙ ВЕРСИЯ 2.0 СУПЕР СИЛЬНАЯ ФИКСАЦИЯ
3,https://basket-04.wbbasket.ru/vol704/part70492...,Консилер для лица 01 LIGHT,70492070,Красота / Корректоры,ООО Софис,7DAYS,55838453,Красота/Макияж/Лицо/Корректор,11650757,31440,...,консилер лицо light,50.0,ДЕКОР,ЛИЦО,Корректор/консилер,Консилер,NaN,7DAYS,Консилер для лица 01 LIGHT,КОНСИЛЕР ДЛЯ ЛИЦА 01 LIGHT
4,https://basket-17.wbbasket.ru/vol2643/part2643...,Адвент календарь с косметикой 12 Days,264347880,Красота / Наборы декоративной косметики,ООО Софис,B.COLOUR PROFESSIONAL,225345435,Красота/Макияж/Лицо/Набор декоративной косметики,11495523,2193,...,адвент календарь косметика days,32.0,ДЕКОР,ГУБЫ,Помада,Стик,NaN,7DAYS,Адвент календарь с косметикой 12 Days,АДВЕНТ КАЛЕНДАРЬ С КОСМЕТИКОЙ 12 DAYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98001,https://basket-17.wbbasket.ru/vol2746/part2746...,Карандаш для губ матовый стойкий 768 оттенок,274636628,Красота / Косметические карандаши,ИП Аскарбеков А Н,NaN,270877652,NaN,50,1,...,карандаш губа матовый стойкий оттенок,37.0,ДЕКОР,ГУБЫ,Карандаш для губ,Автоматический,NaN,НЕТ БРЕНДА,Карандаш для губ матовый стойкий 768 оттенок,КАРАНДАШ ДЛЯ ГУБ МАТОВЫЙ СТОЙКИЙ 768 ОТТЕНОК
98002,https://basket-01.wbbasket.ru/vol89/part8908/8...,Румяна для лица color icon,8908933,Красота / Румяна,АО Единая Европа-Холдинг,Wet n Wild,2646816,NaN,40,1,...,румяна лицо color icon,57.0,ДЕКОР,ЛИЦО,Румяна,Румяна компактные,NaN,WET N WILD,Румяна для лица color icon,РУМЯНА ДЛЯ ЛИЦА COLOR ICON
98003,https://basket-01.wbbasket.ru/vol107/part10791...,Косметический карандаш для губ LP15 тон 07 кла...,10791335,Красота / Косметические карандаши,ООО Евротрейд,LavelleCollection,8162763,NaN,26,1,...,косметический карандаш губа тон классич красный,37.0,ДЕКОР,ГУБЫ,Карандаш для губ,Автоматический,NaN,LAVELLECOLLECTION,Косметический карандаш для губ LP15 тон 07 кла...,КОСМЕТИЧЕСКИЙ КАРАНДАШ ДЛЯ ГУБ LP15 ТОН 07 КЛА...
98004,https://basket-01.wbbasket.ru/vol22/part2268/2...,"Тушь для ресниц ""mega impact"" EC148 тон very b...",2268196,Красота / Туши,АО Единая Европа-Холдинг,Wet n Wild,1967389,NaN,24,1,...,тушь ресница mega impact тон very black,16.0,ДЕКОР,ГЛАЗА,Тушь,Тушь,NaN,WET N WILD,"Тушь для ресниц ""mega impact"" EC148 тон very b...","ТУШЬ ДЛЯ РЕСНИЦ ""MEGA IMPACT"" EC148 ТОН VERY B..."


In [24]:
#удаляем раб колонку
test = test.drop(columns=['Товар'])

In [25]:
# заменяем NaN на 'НЕТ БРЕНДА'
test['Brand'] = test['Brand'].fillna('НЕТ БРЕНДА')

In [26]:
#возвращаем исходное название колонки
test = test.rename(columns={'Товар_old': 'Товар'})

# Сохранение результатов
Результаты сохраняются в файл Excel, который затем можно скачать на локальный компьютер для дальнейшего анализа.

In [27]:
test.to_excel("ДЕКАБРЬ_WB.xlsx", index=False, engine='openpyxl')

In [28]:
try:
    from google.colab import files
    files.download('ДЕКАБРЬ_WB.xlsx')
except ImportError:
    print("Функция загрузки файла не поддерживается в вашей среде")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>